# Aggregation with groupby

In [ ]:
# print all the outputs in a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
pd.set_option('precision', 2) # show only two decimal digits

Load the survey data

In [ ]:
df  = pd.read_csv('cleaned_survey.csv', index_col=0)

In [ ]:
df.head()

## groupby

The method <i>groupby</i> splits the data by the value of a field <i>f</i>. We can then aggregate other columns separately for each value of <i>f</i>

<b>Example</b>: We know how to show the average value of each column.  But groupby allows us to show the average value of each column divided by "Program"

In [ ]:
# we already know to compute the average of each column
df.mean()

In [ ]:
# to do it by Program
gb = df.groupby(by='Program')

In [ ]:
type(gb)

In [ ]:
gb.mean()

#### Example

Display the mean of all columns, grouping by the Job situation

In [ ]:
df.groupby(by='Job').mean()

## Aggregate only some columns

Oftentimes, we don't want to aggregate all columns. For example, we want to find the average of Job grouped by Program.

In [ ]:
df.groupby('Program')['Job'].mean()

Or more columns. For example, we want to find the average of Job, C, and R, grouped by Program.

In [ ]:
df.groupby('Program')[['Job','C','R']].mean()

## Problems

For each Job situation (0=no job, 0.5=part time, 1=full time), find the proportion of students that know SQL

In [ ]:
df.groupby('Job')['SQL'].mean()

For each program, count how many student know SQL.

In [ ]:
df.groupby('Job')['SQL'].sum()

Considering only the students who know SQL, find for each Program the proportion of students who know Java

In [ ]:
df.ix[df.SQL == 1,:].groupby('Program')['Java'].mean()

Which one is faster? Why?
<ol>
<li>df.groupby(by='Program')['SQL'].mean()
<li>df.groupby(by='Program').mean()['SQL']
</ol>

In [ ]:
%timeit df.groupby(by='Program')['SQL'].mean()

In [ ]:
%timeit df.groupby(by='Program').mean()['SQL']

For each Classification skill level, how many MBA students are there? Your result should have 5 rows (one for each classification skill level: 1, 2, 3, 4, and 5)

Here is the wrong way to do it:
<ol>
<li>Keep only the rows of MBA students
<li>Perform group by
</ol>

In [ ]:
df.ix[df.Program == 'MBA'].groupby('Classification').size() #wrong: it does not return one row for each value of Classification

<p>Here is the correct way to do it:</p>
<ol>
<li>Create a dummy variable 'MBA' that has a 1 (or True) if the student is an MBA student and 0 (or False) otherwise
<li>For each classification level, compute the sum of the 'MBA' column. Note that the sum of boolean values counts the True values.
<li>Remove the dummy variable 'MBA'
</ol>

In [ ]:
# create a dummy variable that indicates whether the student is an MBA student
df['MBA'] = df.Program == 'MBA'

In [ ]:
# for each Classification level, sum the values of the variable MBA
df.groupby('Classification')['MBA'].sum()

In [ ]:
# remove dummy variable
df.drop('MBA', axis=1, inplace=True)

## Apply multiple functions (<i>agg</i>)

For each Job situation (0=no job, 0.5=part time, 1=full time), find (1) their number and (2) the proportion of students that know SQL.

In [ ]:
gb = df.groupby('Job')['SQL']

In [ ]:
gb.agg(['mean','size'])

#### Renaming resulting columns

You can pass a dictionary {new column name: function to apply} instead of a list of functions to apply

In [ ]:
gb.agg({'SQL_prop': 'mean','n_students': 'size'})

## Apply multiple arbitrary functions to multiple columns and give them names (agg)

For each Job situation (0=no job, 0.5=part time, 1=full time), compute the average knowledge of SQL, the maximum knowledge of Classification, and the gap between the max and the min Classification score for each Job level

In [ ]:
gb = df.groupby('Job')

In [ ]:
gb.agg({'SQL' : 'mean', 
       'Classification' : ['max', lambda x: x.max() - x.min()]})

We can also give a name to all columns created

In [ ]:
gb.agg({'SQL' : {'SQLmean' : 'mean'}, 
       'Classification' : {'maxClassif':'max',
                           'spreadClassif':lambda x: x.max() - x.min()} })

## group by multiple fields

You can also group by multiple fields. For example, find the mean of all columns grouped by Program and Job situation.

In [ ]:
df.groupby(['Program', 'Job']).mean()

It returns a DataFrame with a <b>Hierarchical Index</b> (i.e., a composite key in a database). In this case, the index is (Program,Job). DataFrames with Hierarchical Indeces are outside the scope of this course because they tend to be hard to deal with; you can avoid them here by using <i>as_index = False</i> inside the <i>groupby</i>. Note: it does not work in all cases.

In [ ]:
df.groupby(['Program', 'Job'],as_index=False).mean()

## Problems

Find the maximum, minimum, and average number of Languages known by students in each Program

In [ ]:
df.groupby('Program')['Languages'].agg(['max','min','mean'])

For each existing combination of programming skills level and Program, report the number of students (call it <i>nStudents</i>) and the proportion that know Python (call it <i>PythonProportion</i>)

In [ ]:
df.columns

In [ ]:
df.groupby(['ProgSkills','Program']).agg({
    'Job' : {'nStudents': 'size'},
    'Python' : {'PythonProportion' : 'mean'}
    })

HARD. For each Program, report:
<ul>
<li>the number of students who know both Python and C (call it <i>C_Python_Students</i>, and note that it can be equal to 0)
<li>the gap between max and mean Clustering knowledge (call it <i>CluGap</i>)
</ul>

In [ ]:
df['PythonAndC'] = (df.Python == 1) & (df.C == 1)

In [ ]:
df.groupby('Program').agg({
        'PythonAndC' : {'C_Python_Students' : 'sum'},
        'Clustering' : {'CluGap' : lambda x : x.max() - x.mean()}
    })

In [ ]:
df.drop('PythonAndC', axis = 1 , inplace=True)

## Advanced: retrieve unaggregated rows (<i>apply</i>)

Sometimes, for each group-by value we want to retrieve one or more rows. For example, for each program report the student who knows most languages (report more than one students in case of ties)

In [ ]:
df.groupby('Program').apply(lambda d : d.ix[d.Languages == d.Languages.max(),:])

## Problems

For each ProgSkills level, find whether the student (or students in case of ties) with the highest Classification skills know C and Java

In [ ]:
df.groupby('ProgSkills').apply(lambda d : d.ix[d.Classification == d.Classification.max(),['C','Java']])

For each ProgSkills level, find the Program with most students that have that ProgSkill level

In [ ]:
df.groupby('ProgSkills').apply(lambda d : d.groupby('Program').size().nlargest(1))